In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210206.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],Rank your movie choices. You don't have to do a full ranking. [Seventh Choice],Rank your movie choices. You don't have to do a full ranking. [Eighth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice],What theme should we explore next week? [Eighth Choice],What theme should we explore next week? [Ninth Choice]
0,2/2/2021 17:05:30,Monkeybone (2001),Pawn Shop Chronicles (2013),With Honors (1994),Gods and Monsters (1998),School Ties (1992),Furry Vengeance (2010),Still Breathing (1997),George of the Jungle (1997),"GOLAN-GLOBUS A-GO-GO: THE CANNON GROUP, INC.",MICHAEL CAINE/MY COCAINE,INTERNATIONAL SCIENCE FICTION,MEXICAN FILMS IN MEXICAN LANGUAGES,D'YA LIKE DAGS,DOCUMENTARY! NOW!,WHAT IF I JUST SHOT EVERYBODY?,[Default to Nicolas Cage],KNOCKOFFS OF ANIMATED HITS
1,2/2/2021 17:07:45,School Ties (1992),Pawn Shop Chronicles (2013),Gods and Monsters (1998),Furry Vengeance (2010),George of the Jungle (1997),Monkeybone (2001),Still Breathing (1997),With Honors (1994),MEXICAN FILMS IN MEXICAN LANGUAGES,MICHAEL CAINE/MY COCAINE,WHAT IF I JUST SHOT EVERYBODY?,INTERNATIONAL SCIENCE FICTION,"GOLAN-GLOBUS A-GO-GO: THE CANNON GROUP, INC.",D'YA LIKE DAGS,DOCUMENTARY! NOW!,[Default to Nicolas Cage],KNOCKOFFS OF ANIMATED HITS
2,2/2/2021 17:09:00,Gods and Monsters (1998),Pawn Shop Chronicles (2013),Still Breathing (1997),School Ties (1992),Monkeybone (2001),Furry Vengeance (2010),With Honors (1994),NaN,MICHAEL CAINE/MY COCAINE,D'YA LIKE DAGS,"GOLAN-GLOBUS A-GO-GO: THE CANNON GROUP, INC.",INTERNATIONAL SCIENCE FICTION,DOCUMENTARY! NOW!,WHAT IF I JUST SHOT EVERYBODY?,KNOCKOFFS OF ANIMATED HITS,MEXICAN FILMS IN MEXICAN LANGUAGES,[Default to Nicolas Cage]
3,2/2/2021 17:09:57,Monkeybone (2001),Gods and Monsters (1998),NaN,NaN,NaN,NaN,NaN,NaN,"GOLAN-GLOBUS A-GO-GO: THE CANNON GROUP, INC.",INTERNATIONAL SCIENCE FICTION,MEXICAN FILMS IN MEXICAN LANGUAGES,WHAT IF I JUST SHOT EVERYBODY?,NaN,NaN,NaN,NaN,NaN
4,2/2/2021 17:26:23,Monkeybone (2001),Furry Vengeance (2010),George of the Jungle (1997),NaN,NaN,NaN,NaN,NaN,INTERNATIONAL SCIENCE FICTION,DOCUMENTARY! NOW!,MEXICAN FILMS IN MEXICAN LANGUAGES,WHAT IF I JUST SHOT EVERYBODY?,MICHAEL CAINE/MY COCAINE,"GOLAN-GLOBUS A-GO-GO: THE CANNON GROUP, INC.",KNOCKOFFS OF ANIMATED HITS,NaN,NaN
5,2/2/2021 18:24:21,School Ties (1992),With Honors (1994),Monkeybone (2001),Still Breathing (1997),Gods and Monsters (1998),Pawn Shop Chronicles (2013),George of the Jungle (1997),Furry Vengeance (2010),D'YA LIKE DAGS,MEXICAN FILMS IN MEXICAN LANGUAGES,INTERNATIONAL SCIENCE FICTION,MICHAEL CAINE/MY COCAINE,WHAT IF I JUST SHOT EVERYBODY?,KNOCKOFFS OF ANIMATED HITS,[Default to Nicolas Cage],DOCUMENTARY! NOW!,"GOLAN-GLOBUS A-GO-GO: THE CANNON GROUP, INC."
6,2/2/2021 18:44:19,Monkeybone (2001),Gods and Monsters (1998),George of the Jungle (1997),Still Breathing (1997),With Honors (1994),School Ties (1992),Pawn Shop Chronicles (2013),Furry Vengeance (2010),DOCUMENTARY! NOW!,[Default to Nicolas Cage],INTERNATIONAL SCIENCE FICTION,MEXICAN FILMS IN MEXICAN LANGUAGES,MICHAEL CAINE/MY COCAINE,WHAT IF I JUST SHOT EVERYBODY?,KNOCKOFFS OF ANIMATED HITS,D'YA LIKE DAGS,"GOLAN-GLOBUS A-GO-GO: THE CANNON GROUP, INC."
7,2/2/2021 18:45:42,Monkeyb

In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Monkeybone (2001)', 'Pawn Shop Chronicles (2013)',
       'With Honors (1994)', 'Gods and Monsters (1998)',
       'School Ties (1992)', 'Furry Vengeance (2010)',
       'Still Breathing (1997)', 'George of the Jungle (1997)'],
      dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Monkeybone (2001)': <Candidate('Monkeybone (2001)')>,
 'Pawn Shop Chronicles (2013)': <Candidate('Pawn Shop Chronicles (2013)')>,
 'With Honors (1994)': <Candidate('With Honors (1994)')>,
 'Gods and Monsters (1998)': <Candidate('Gods and Monsters (1998)')>,
 'School Ties (1992)': <Candidate('School Ties (1992)')>,
 'Furry Vengeance (2010)': <Candidate('Furry Vengeance (2010)')>,
 'Still Breathing (1997)': <Candidate('Still Breathing (1997)')>,
 'George of the Jungle (1997)': <Candidate('George of the Jungle (1997)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Monkeybone (2001), Pawn Shop Chronicles (2013), With Honors (1994), Gods and Monsters (1998), School Ties (1992), Furry Vengeance (2010), Still Breathing (1997), George of the Jungle (1997))>,
 <Ballot(School Ties (1992), Pawn Shop Chronicles (2013), Gods and Monsters (1998), Furry Vengeance (2010), George of the Jungle (1997), Monkeybone (2001), Still Breathing (1997), With Honors (1994))>,
 <Ballot(Gods and Monsters (1998), Pawn Shop Chronicles (2013), Still Breathing (1997), School Ties (1992), Monkeybone (2001), Furry Vengeance (2010), With Honors (1994))>,
 <Ballot(Monkeybone (2001), Gods and Monsters (1998))>,
 <Ballot(Monkeybone (2001), Furry Vengeance (2010), George of the Jungle (1997))>,
 <Ballot(School Ties (1992), With Honors (1994), Monkeybone (2001), Still Breathing (1997), Gods and Monsters (1998), Pawn Shop Chronicles (2013), George of the Jungle (1997), Furry Vengeance (2010))>,
 <Ballot(Monkeybone (2001), Gods and Monsters (1998), George of the Jungle (1997),

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

FINAL RESULT
Candidate                      Votes  Status
---------------------------  -------  --------
Monkeybone (2001)                  6  Elected
School Ties (1992)                 2  Rejected
Gods and Monsters (1998)           1  Rejected
Furry Vengeance (2010)             1  Rejected
Pawn Shop Chronicles (2013)        0  Rejected
With Honors (1994)                 0  Rejected
George of the Jungle (1997)        0  Rejected
Still Breathing (1997)             0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                                       Votes  Status
--------------------------------------------  -------  --------
MEXICAN FILMS IN MEXICAN LANGUAGES                  2  Hopeful
DOCUMENTARY! NOW!                                   2  Hopeful
GOLAN-GLOBUS A-GO-GO: THE CANNON GROUP, INC.        2  Hopeful
MICHAEL CAINE/MY COCAINE                            1  Hopeful
D'YA LIKE DAGS                                      1  Hopeful
INTERNATIONAL SCIENCE FICTION                       1  Hopeful
WHAT IF I JUST SHOT EVERYBODY?                      1  Rejected
[Default to Nicolas Cage]                           0  Rejected
KNOCKOFFS OF ANIMATED HITS                          0  Rejected

ROUND 2
Candidate                                       Votes  Status
--------------------------------------------  -------  --------
INTERNATIONAL SCIENCE FICTION                       2  Hopeful
MEXICAN FILMS IN MEXICAN LANGUAGES                  2  Hopeful
DOCUMENTARY! NOW!                  